In [138]:
import numpy as np
from emo_utils import *
import emoji
import matplotlib.pyplot as plt
import re

%matplotlib inline

In [139]:
#X_test, Y_test = read_csv('data/data-English/train.csv')
X_test, Y_test = read_csv('data/data-English/test.csv')

X_train, Y_train = read_csv('data/data-English/train.csv')

In [140]:
def category(label):
    a=['NEGATIVE','POSITIVE','SUGGESTION']
    return a[label-1]
print(category(1))


NEGATIVE


In [141]:
maxLen = len(max(X_train, key=len).split())

In [142]:
'''
for i in range (1,324):
    A=re.split(r". ;| |!|,|\?",X_train[i])
    if '' in A:
        print(i)
        print(A)
    i+=1
'''
index = 5
#print(X_train[index], label_to_emoji(Y_train[index]))
print(X_train[index], category(Y_train[index]))
symbol="!,"
s=re.split(r". ;| |!|,|\?",X_train[index])
print(s)

This course is more of an overview of what is to come NEGATIVE
['This', 'course', 'is', 'more', 'of', 'an', 'overview', 'of', 'what', 'is', 'to', 'come']


In [143]:
Y_oh_train = convert_to_one_hot(Y_train, C = 3)
Y_oh_test = convert_to_one_hot(Y_test, C = 3)

In [144]:
index =109
print(Y_train[index], "is converted into one hot", Y_oh_train[index])

1 is converted into one hot [0. 1. 0.]


In [145]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')
#读一个一行 一个词+50维
#print(word_to_vec_map['it']) --这个是字典，对应的词和它的词向量

In [146]:
word = "specialization"
index = 289846
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(index) + "th word in the vocabulary is", index_to_word[index])
print(word_to_vec_map[word])

the index of specialization in the vocabulary is 339014
the 289846th word in the vocabulary is potatos
[-0.53862   0.46822  -0.91289  -0.37902  -0.34028   0.33678   0.42091
 -0.6817    0.59982  -0.048546  0.74597  -0.25944   0.50634  -0.059825
 -0.85124   0.21286   0.17618   0.50062   0.62398   0.14575   0.31584
 -0.33425  -0.19406   0.18952  -0.21367   0.33764  -0.20979  -0.51248
  0.033012  0.52382   1.2516    0.36535  -0.10563  -0.95343   0.3965
  0.63945  -0.75734   1.0288    0.3788    0.59807  -0.4078   -0.5924
 -0.097769  0.88356  -0.49438  -0.819     1.5071    0.93786  -0.17713
  0.20902 ]


In [147]:
import numpy as np
np.random.seed(0)
from keras.models import Model
#from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers import Dense, Input, Dropout, Activation
from keras.layers.recurrent import LSTM

from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

In [148]:
# GRADED FUNCTION: sentences_to_indices

def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    m = X.shape[0]                                   # number of training examples
    ### START CODE HERE ###
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m, max_len))
    
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        #sentence_words =X[i].lower().split()
        
        sentence_words=re.split(r";| |!|,|\?",X[i].lower())


        # Initialize j to 0
        j = 0
        
        # Loop over the words of sentence_words
        for w in sentence_words:
            if w=='':
                continue
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            try:
                X_indices[i, j] = word_to_index[w]
                # Increment j to j + 1
                j = j+1
            except:
                continue
            
    ### END CODE HERE ###
    
    return X_indices

#词在词向量里面的位置

In [149]:
X1 = np.array(["student", "I hate it", "this teacher is enthusiastic"])
X1_indices = sentences_to_indices(X1,word_to_index, max_len = 5)
print("X1 =", X1)
print("X1_indices =", X1_indices)

X1 = ['student' 'I hate it' 'this teacher is enthusiastic']
X1_indices = [[345140.      0.      0.      0.      0.]
 [185457. 174339. 193716.      0.      0.]
 [358160. 354429. 192973. 137919.      0.]]


In [150]:
# GRADED FUNCTION: pretrained_embedding_layer

def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [151]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
#print(type(embedding_layer))
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][49])
#这个应该是指的把词向量变成了一个矩阵，比如5000*50 的矩阵

weights[0][1][3] = 0.78858


In [152]:
# GRADED FUNCTION: Emojify_V2

def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(input_shape, dtype='int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)   
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(128,return_sequences=True)(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.4)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128, return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.4)(X)
    
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(3)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices, outputs=X)
    
    ### END CODE HERE ###
    
    return model

In [153]:
model = Emojify_V2((maxLen,), word_to_vec_map, word_to_index)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 52)                0         
_________________________________________________________________
embedding_14 (Embedding)     (None, 52, 50)            20000050  
_________________________________________________________________
lstm_13 (LSTM)               (None, 52, 128)           91648     
_________________________________________________________________
dropout_13 (Dropout)         (None, 52, 128)           0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_14 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 3)                 387       
__________

In [154]:
model.compile(loss='categorical_crossentropy', optimizer='Adamax', metrics=['accuracy'])

In [155]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 3)
print(maxLen)

52


In [178]:
model.fit(X_train_indices, Y_train_oh, epochs = 1, batch_size = 32, shuffle=True)

Epoch 1/1
477/477 [==============================] - 7s - loss: 0.0223 - acc: 0.9958     


In [179]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
#得到了测试集的每句话每个词的位置
Y_test_oh = convert_to_one_hot(Y_test, C = 3)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)

115/115 [==============================] - 0s     

Test accuracy =  0.756521739648736


In [67]:
# This code allows you to see the mislabelled examples
C = 3
y_test_oh = np.eye(C)[Y_test.reshape(-1)]
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        print('Expected emoji:'+ category(Y_test[i]) + ' prediction: '+ X_test[i] + category(num).strip())

Expected emoji:NEGATIVE prediction: really enjoyed the writing assignments as a creative project but I do not feel I was able to fully grasp the concepts despite getting good marksPOSITIVE
Expected emoji:NEGATIVE prediction: The feedback we receive from peers is usually limited to a couple of words and does not really help you assess how well you did	SUGGESTION
Expected emoji:NEGATIVE prediction: Not useful in my opinionPOSITIVE
Expected emoji:NEGATIVE prediction: I got a bit impatient with that towards the end	1POSITIVE
Expected emoji:NEGATIVE prediction: I wish it was more into a solo topic It would be great if it was given only git/github lectures so the students can be more comfortable when it comes to uploading fles in the following coursesSUGGESTION
Expected emoji:NEGATIVE prediction: Really easy coursePOSITIVE
Expected emoji:NEGATIVE prediction: Very basic stuff - if you've ever done any coding before and have even a rudimentary knowledge of Github - you can skip this courseSUGG

In [22]:
# Change the sentence below to see your prediction. Make sure all the words are in the Glove embeddings.  
x_test = np.array(['it is not good'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
#print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))
print(x_test[0])
print(category(np.argmax(model.predict(X_test_indices))))

it is not good
NEGATIVE
